# 1. Thư viện

In [1]:
from elasticsearch import Elasticsearch, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

# 2. Khởi tạo ElasticSearch Client

In [2]:
client = Elasticsearch(
    "https://localhost:9200",
    ca_certs = r"C:\ElasticSearch\elasticsearch-8.15.1\config\certs\http_ca.crt",
    basic_auth = ("elastic", "taFBkRBmF3-OR93OQ*nF"),
    request_timeout = None
)

# API key should have cluster monitor rights
client.info()

ObjectApiResponse({'name': 'DESKTOP-B86U75E', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'hYbYS8ZRT8uazPw8fPhRfA', 'version': {'number': '8.15.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '253e8544a65ad44581194068936f2a5d57c2c051', 'build_date': '2024-09-02T22:04:47.310170297Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [3]:
print(client.ping())

True


# 3. Tạo interface endpoint

In [4]:
# Link Document: https://www.elastic.co/guide/en/elasticsearch/reference/current/delete-inference-api.html

try:
    client.inference.delete(inference_id="my-elser-endpoint")
except exceptions.NotFoundError:
    # Inference endpoint does not exist
    print("Inference endpoint does not exist")
    pass

Inference endpoint does not exist


In [5]:
# Link Docs:
# - Tạo Inference: https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html
# - Tạo Inference với ELSER – a retrieval model trained by Elastic: https://www.elastic.co/guide/en/elasticsearch/reference/current/infer-service-elser.html

try:
    client.options(
        request_timeout=120, max_retries=3, retry_on_timeout=True
    ).inference.put(
        task_type = "sparse_embedding",
        inference_id = "my-elser-endpoint",
        inference_config = {
            "service": "elser",
            "service_settings": {
                "num_allocations": 1, 
                "num_threads": 1
            },
        },
    )
    print("Inference endpoint created successfully")
except exceptions.BadRequestError as e:
    if e.error == "resource_already_exists_exception":
        print("Inference endpoint created successfully")
    else:
        raise e

AuthorizationException: AuthorizationException(403, 'security_exception', 'current license is non-compliant for [ml]')